In [90]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time 
import pandas as pd
import re

In [92]:
start = time.time()
# Initialize the webdriver
# de manière basique
# driver = webdriver.Chrome()

# avec ChromeDriverManager mieux car update du driver automatiquement
driver = webdriver.Chrome(ChromeDriverManager().install())

# si on a télécharger le driver directement dans le dossier 
# service = Service(executable_path="./chromedriver.exe")
# driver = webdriver.Chrome(service=service)

# Navigate to the website
# driver.get("https://www.billboard.com/charts/hot-100/")
driver.get("https://www.billboard.com/charts/billboard-global-200/")
# driver.get("https://www.billboard.com/charts/billboard-200/")

# Wait for the page to load
# driver.implicitly_wait(10)
time.sleep(5)

# reject all cookies
driver.find_element(By.XPATH,"/html/body/div[5]/div[2]/div/div/div[2]/div/div/button[1]").click()

iterateur100 = list(range(1,110,11))
iterateur100.remove(1)
iterateur200 = list(range(1,220,11))
iterateur200.remove(1)

time.sleep(50)
# nb_max 111 ou 221
def scraper(list,xpath,iterateur,nb_max):
    for i in range(2,nb_max):
        if i not in iterateur : 
            WebElement = driver.find_element(By.XPATH,xpath%(i))
            list.append(WebElement.text)

    return list

XpathTitle = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/h3"
XpathArtist = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/span"
XpathRank = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[1]/span "
XpathLastWeek = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[4]/span"
XpathPeakPosition = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[5]/span"
XpathWeeksOnChart = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[6]/span"

allXpath = [XpathTitle,XpathArtist,XpathRank,XpathLastWeek,XpathPeakPosition,XpathWeeksOnChart]

title,artist,rank,last_week,peak_pos,weeks_on_chart = [],[],[],[],[],[]

allList = [title,artist,rank,last_week,peak_pos,weeks_on_chart]

i = [scraper(i,j,iterateur200,221) for i, j in zip(allList, allXpath)]

# for i, j in zip(allList, allXpath):
#     i = scraper(i,j,iterateur200,221)

df = pd.DataFrame(list(zip(title,artist,rank,last_week,peak_pos,weeks_on_chart)),columns=['Title','Artist','Rank','Last Week','Peak Positon','Weeks on charts'])
print(df)

# Close the webdriver
driver.close()

end = time.time()
temps = end - start
print("temps d'execution :",temps,"s")

C:\Users\thoma\AppData\Local\Temp\ipykernel_15452\1624214680.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


                            Title                        Artist Rank  \
0                         Flowers                   Miley Cyrus    1   
1                       Kill Bill                           SZA    2   
2    Bzrp Music Sessions, Vol. 53            Bizarrap & Shakira    3   
3                          Unholy        Sam Smith & Kim Petras    4   
4                       Calm Down           Rema & Selena Gomez    5   
..                            ...                           ...  ...   
195                    In The End                   Linkin Park  196   
196                       Ferrari  James Hype & Miggy Dela Rosa  197   
197          Just The Way You Are                    Bruno Mars  198   
198              Call Out My Name                    The Weeknd  199   
199                     Good Days                           SZA  200   

    Last Week Peak Positon Weeks on charts  
0           1            1               4  
1           3            1               9  


In [93]:
song_artists = list(zip(df['Artist'],df["Title"]))
song_artists

[('Miley Cyrus', 'Flowers'),
 ('SZA', 'Kill Bill'),
 ('Bizarrap & Shakira', 'Bzrp Music Sessions, Vol. 53'),
 ('Sam Smith & Kim Petras', 'Unholy'),
 ('Rema & Selena Gomez', 'Calm Down'),
 ('Metro Boomin, The Weeknd & 21 Savage', "Creepin'"),
 ('Harry Styles', 'As It Was'),
 ('RAYE Featuring 070 Shake', 'Escapism'),
 ('David Guetta & Bebe Rexha', "I'm Good (Blue)"),
 ('Taylor Swift', 'Anti-Hero'),
 ('Morgan Wallen', 'Last Night'),
 ('NewJeans', 'OMG'),
 ('Miguel', 'Sure Thing'),
 ('NewJeans', 'Ditto'),
 ('PinkPantheress & Ice Spice', "Boy's A Liar, Pt. 2"),
 ('Yandel & Feid', 'Yandel 150'),
 ('Manuel Turizo', 'La Bachata'),
 ('Stephen Sanchez', 'Until I Found You'),
 ('JVKE', 'Golden Hour'),
 ('The Weeknd', 'Die For You'),
 ('Meghan Trainor', 'Made You Look'),
 ('Arcangel & Bad Bunny', 'La Jumpa'),
 ('Beyonce', 'Cuff It'),
 ('OneRepublic', "I Ain't Worried"),
 ('The Weeknd & Ariana Grande', 'Save Your Tears'),
 ('Tom Odell', 'Another Love'),
 ('Drake & 21 Savage', 'Rich Flex'),
 ('SZA',

In [94]:
# #modification du nom de l'artiste pour la recherche Genius
for i in range(len(song_artists)):
    
    if 'X' in song_artists[i][0] and 'Lil Nas X' not in song_artists[i][0]:
        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace("X", "&")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

('Fuerza Regida & Grupo Frontera', 'Bebe Dame')
('Peso Pluma & Natanael Cano', 'PRC')
('Carin Leon & Grupo Frontera', 'Que Vuelvas')
('Junior H & Oscar Maydon', 'Fin de Semana')
('Imagine Dragons & JID', 'Enemy')
('TOMORROW & TOGETHER', 'Sugar Rush Ride')
('Fuerza Regida & Natanael Cano', 'Ch y La Pizza')


In [95]:
for i in range(len(song_artists)):

    if 'vs.' in song_artists[i][0] :

        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace("vs.", "")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

print(song_artists)

('Lana Del Rey  Cedric Gervais', 'Summertime Sadness')
[('Miley Cyrus', 'Flowers'), ('SZA', 'Kill Bill'), ('Bizarrap & Shakira', 'Bzrp Music Sessions, Vol. 53'), ('Sam Smith & Kim Petras', 'Unholy'), ('Rema & Selena Gomez', 'Calm Down'), ('Metro Boomin, The Weeknd & 21 Savage', "Creepin'"), ('Harry Styles', 'As It Was'), ('RAYE Featuring 070 Shake', 'Escapism'), ('David Guetta & Bebe Rexha', "I'm Good (Blue)"), ('Taylor Swift', 'Anti-Hero'), ('Morgan Wallen', 'Last Night'), ('NewJeans', 'OMG'), ('Miguel', 'Sure Thing'), ('NewJeans', 'Ditto'), ('PinkPantheress & Ice Spice', "Boy's A Liar, Pt. 2"), ('Yandel & Feid', 'Yandel 150'), ('Manuel Turizo', 'La Bachata'), ('Stephen Sanchez', 'Until I Found You'), ('JVKE', 'Golden Hour'), ('The Weeknd', 'Die For You'), ('Meghan Trainor', 'Made You Look'), ('Arcangel & Bad Bunny', 'La Jumpa'), ('Beyonce', 'Cuff It'), ('OneRepublic', "I Ain't Worried"), ('The Weeknd & Ariana Grande', 'Save Your Tears'), ('Tom Odell', 'Another Love'), ('Drake & 21 Sa

In [96]:
#modification du nom de l'artiste pour la recherche Genius

for i in range(len(song_artists)):

    if " With " in song_artists[i][0] :

        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace(" With ", " ")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

print(song_artists)

('Kane Brown Katelyn Brown', 'Thank God')
[('Miley Cyrus', 'Flowers'), ('SZA', 'Kill Bill'), ('Bizarrap & Shakira', 'Bzrp Music Sessions, Vol. 53'), ('Sam Smith & Kim Petras', 'Unholy'), ('Rema & Selena Gomez', 'Calm Down'), ('Metro Boomin, The Weeknd & 21 Savage', "Creepin'"), ('Harry Styles', 'As It Was'), ('RAYE Featuring 070 Shake', 'Escapism'), ('David Guetta & Bebe Rexha', "I'm Good (Blue)"), ('Taylor Swift', 'Anti-Hero'), ('Morgan Wallen', 'Last Night'), ('NewJeans', 'OMG'), ('Miguel', 'Sure Thing'), ('NewJeans', 'Ditto'), ('PinkPantheress & Ice Spice', "Boy's A Liar, Pt. 2"), ('Yandel & Feid', 'Yandel 150'), ('Manuel Turizo', 'La Bachata'), ('Stephen Sanchez', 'Until I Found You'), ('JVKE', 'Golden Hour'), ('The Weeknd', 'Die For You'), ('Meghan Trainor', 'Made You Look'), ('Arcangel & Bad Bunny', 'La Jumpa'), ('Beyonce', 'Cuff It'), ('OneRepublic', "I Ain't Worried"), ('The Weeknd & Ariana Grande', 'Save Your Tears'), ('Tom Odell', 'Another Love'), ('Drake & 21 Savage', 'Rich 

In [97]:
#modification du nom de l'artiste pour la recherche Genius
for i in range(len(song_artists)):

    if "Coolio Featuring L.V." in song_artists[i][0] :

        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace("Coolio Featuring L.V.", "Colio")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

('Colio', "Gangsta's Paradise")


In [98]:
#modification du nom de l'artiste pour la recherche Genius
for i in range(len(song_artists)):

    if "Quevedo & Mike Towers" in song_artists[i][0] :

        artist_list = list(song_artists[i])
        artist_list[0] = artist_list[0].replace("Quevedo & Mike Towers", "Quevedo")
        song_artists[i] = tuple(artist_list)
        print(song_artists[i])

('Quevedo', 'Playa del Ingles')


In [99]:
song_artists.remove(('Carin Leon & Grupo Frontera', 'Que Vuelvas'))
print(('Carin Leon & Grupo Frontera', 'Que Vuelvas') in song_artists)

False


In [156]:
driver_genius = webdriver.Chrome(ChromeDriverManager().install())

    # ouvrir le site et accepter les cookies

driver_genius.get("https://genius.com/")
driver_genius.implicitly_wait(10)
driver_genius.find_element(By.ID,"onetrust-accept-btn-handler").click()
time.sleep(3)
 
song,artist,date,producteurs_text,ecrivains_text,publisher_text,distributor_text,Tags = [],[],[],[],[],[],[],[]


for i in range(100):

    driver_genius.get("https://genius.com/")

    # activer la recherche
    element = driver_genius.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/form/input")
    element.send_keys(song_artists[i][1]+' '+song_artists[i][0], Keys.ENTER)
    
    song.append(song_artists[i][0])
    artist.append(song_artists[i][1])

    # clicker sur la chanson
    try :

        driver_genius.find_element(By.XPATH,"/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a/div[2]").click()

    
    except:

        driver_genius.find_element(By.XPATH,"/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[2]/search-result-section/div/div[2]/search-result-items/div[1]/search-result-item/div/mini-song-card/a").click()

    #scroll to credits

    target_credits = driver_genius.find_element(By.CSS_SELECTOR,"div.SongInfo__Title-nekw6x-1")
    driver_genius.execute_script("arguments[0].scrollIntoView();", target_credits)

    time.sleep(5)


    # Si le bouton expand est présent

    try: 
        #click on expand if this button exist

        all_elements_after_target_credit = target_credits.find_elements(By.XPATH, ".//following::*")

        # trouve le 1er element dont le tag_name est "button"

        for element in all_elements_after_target_credit:
            if element.tag_name == "button":
                element.click()
                break

        driver_genius.implicitly_wait(10)

        # Localiser la balise "div" contenant la date en utilisant son contenu texte

        date_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Release Date')]")
        

        # Extraire la date à partir du texte en utilisant des expressions régulières
        if date_element:
            date_text = date_element.find_element(By.XPATH,'./following-sibling::div').text
            date_regex = r"\w+\s+\d{1,2},\s+\d{4}"
            match = re.search(date_regex, date_text)
            if match:

                date.append(match.group())

        else:
                
            date.append("Nan")

        time.sleep(2)

        #Recherche des producteurs,ecrivains,publisher,distributor et tags

        ##################################################PRODUCTEURS#############################################
        try:
            producteurs_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Produced By')]")
            producteurs_text.append(producteurs_element.find_element(By.XPATH,'./following-sibling::div').text)
        except:
            producteurs_text.append("Nan")
        ############################################ECRIVAIN#############################################
        time.sleep(2)
        try:
            ecrivains_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Written By')]")
            ecrivains_text.append(ecrivains_element.find_element(By.XPATH,'./following-sibling::div').text)
        except:  
            ecrivains_text.append("Nan")
        ########################################PUBLISHER##################################################
        time.sleep(2)
        try:
            publisher_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Publisher')]")
            publisher_text.append(publisher_element.find_element(By.XPATH,'./following-sibling::div').text)
        except:
            publisher_text.append("Nan")
        #######################################DISTIBUTEURS###############################################
        time.sleep(2)
        try:
            distributor_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Distributor')]")
            distributor_text.append(distributor_element.find_element(By.XPATH,'./following-sibling::div').text)
        except:
            distributor_text.append("Nan")

        #############################################TAGS#####################################################
        time.sleep(2)

        try:
            tag_pos = driver_genius.find_element(By.CLASS_NAME,"SongTags__Title-xixwg3-0")
            driver_genius.execute_script("arguments[0].scrollIntoView();", tag_pos)
            name = driver_genius.find_elements(By.CLASS_NAME, "SongTags__Container-xixwg3-1")

            
            tag=[]

            for elt in name:

                tag.append(elt.text)

            tags = ''.join(set(tag))  
            
            Tags.append(tags.replace('\n',', '))
        
        except:

            Tags.append("Nan")


    # Si le bouton expand n'est pas présent

    except: 
        
        

        # Localiser la balise "div" contenant la date en utilisant son contenu texte

        date_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Release Date')]")

        # Extraire la date à partir du texte en utilisant des expressions régulières
        if date_element:
            date_text = date_element.find_element(By.XPATH,'./following-sibling::div').text
            date_regex = r"\w+\s+\d{1,2},\s+\d{4}"
            match = re.search(date_regex, date_text)
            if match:
                date.append(match.group())

        else: 

            date.append("Nan")

        time.sleep(2)

            #Recherche des producteurs,ecrivains,publisher,distributor et tags

        ##################################################PRODUCTEURS#############################################
        time.sleep(2)

        try:
            producteurs_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Produced By')]")
            producteurs_text.append(producteurs_element.find_element(By.XPATH,'./following-sibling::div').text)
        except:
            producteurs_text.append("Nan")
        ############################################ECRIVAIN#############################################
        time.sleep(2)

        try:
            ecrivains_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Written By')]")
            ecrivains_text.append(ecrivains_element.find_element(By.XPATH,'./following-sibling::div').text)
        except:  
            ecrivains_text.append("Nan")
        ########################################PUBLISHER##################################################
        time.sleep(2)

        try:
            publisher_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Publisher')]")
            publisher_text.append(publisher_element.find_element(By.XPATH,'./following-sibling::div').text)
        except:
            publisher_text.append("Nan")
        #######################################DISTIBUTEURS###############################################
        time.sleep(2)

        try:
            distributor_element = driver_genius.find_element(By.XPATH,"//div[contains(text(),'Distributor')]")
            distributor_text.append(distributor_element.find_element(By.XPATH,'./following-sibling::div').text)
        except:
            distributor_text.append("Nan")

        #############################################TAGS#####################################################
        time.sleep(2)
        try:
            tag_pos = driver_genius.find_element(By.CLASS_NAME,"SongTags__Title-xixwg3-0")
            driver_genius.execute_script("arguments[0].scrollIntoView();", tag_pos)
            name = driver_genius.find_elements(By.CLASS_NAME, "SongTags__Container-xixwg3-1")

            
            tag=[]

            for elt in name:

                tag.append(elt.text)

            tags = ''.join(set(tag))  
            
            Tags.append(tags.replace('\n',', '))
        
        except:

            Tags.append("Nan")

        # on revient à l'accueil

        driver_genius.get("https://genius.com/")

    print(i)
print(date, producteurs_text,ecrivains_text,publisher_text,distributor_text,Tags)
# df_genius = pd.DataFrame( {'Realise Date': date, 'Producers': producteurs_text, 'Writes' : ecrivains_text ,'Publisher':publisher_text,'Distributors':distributor_text, 'Tags':distributor_text})

C:\Users\thoma\AppData\Local\Temp\ipykernel_15452\3285826286.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_genius = webdriver.Chrome(ChromeDriverManager().install())


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
['January 13, 2023', 'December 9, 2022', 'January 11, 2023', 'September 22, 2022', 'August 26, 2022', 'December 2, 2022', 'April 1, 2022', 'October 12, 2022', 'July 31, 2012', 'October 21, 2022', 'January 31, 2023', 'January 2, 2023', 'November 30, 2010', 'December 19, 2022', 'February 3, 2023', 'December 20, 2022', 'May 26, 2022', 'September 1, 2021', 'July 15, 2022', 'November 25, 2016', 'October 21, 2022', 'November 30, 2022', 'July 29, 2022', 'May 13, 2022', 'April 23, 2021', 'October 15, 2012', 'November 4, 2022', 'December 9, 2022', 'December 16, 2022', 'February 2, 2023', 'October 7, 2022', 'October 4, 2019', 'October 17, 2022', 'November 30, 2022', 'July 6, 2022', 'August 5, 2022', 'November 2

In [157]:
print(len(date))
print(len(producteurs_text))
print(len(ecrivains_text))
print(len(publisher_text))
print(len(distributor_text))
print(len(Tags))

97
100
100
100
100
100


In [158]:
print(date)

['January 13, 2023', 'December 9, 2022', 'January 11, 2023', 'September 22, 2022', 'August 26, 2022', 'December 2, 2022', 'April 1, 2022', 'October 12, 2022', 'July 31, 2012', 'October 21, 2022', 'January 31, 2023', 'January 2, 2023', 'November 30, 2010', 'December 19, 2022', 'February 3, 2023', 'December 20, 2022', 'May 26, 2022', 'September 1, 2021', 'July 15, 2022', 'November 25, 2016', 'October 21, 2022', 'November 30, 2022', 'July 29, 2022', 'May 13, 2022', 'April 23, 2021', 'October 15, 2012', 'November 4, 2022', 'December 9, 2022', 'December 16, 2022', 'February 2, 2023', 'October 7, 2022', 'October 4, 2019', 'October 17, 2022', 'November 30, 2022', 'July 6, 2022', 'August 5, 2022', 'November 26, 2016', 'January 27, 2023', 'November 24, 2022', 'July 17, 2022', 'November 3, 2022', 'May 6, 2022', 'April 22, 2022', 'June 29, 2022', 'January 24, 2023', 'May 6, 2022', 'December 8, 2022', 'July 9, 2021', 'November 29, 2019', 'June 29, 2020', 'October 28, 2022', 'June 26, 2017', 'March

In [159]:
for i in range(len(date)):

    if date[i]=="Nan":

        print(i)

In [154]:
print(song_artists[28])

('Fuerza Regida & Grupo Frontera', 'Bebe Dame')


In [155]:
df_genius_100 = pd.DataFrame( { 'song':song, 'artist':artist,'date':date, 'Producers': producteurs_text, 'Writes' : ecrivains_text ,'Publisher':publisher_text,'Distributors':distributor_text,'Tags':Tags})
df_genius_100

,song,artist,date,Producers,Writes,Publisher,Distributors,Tags
0,Miley Cyrus,Flowers,"January 13, 2023",Kid Harpoon & Tyler Johnson,"Miley Cyrus, Aldae & Michael Pollack","Pure Tone Music, Suga Bear Recordz, Warner-Tam...",Nan,"Pop, Funk-Pop, Dance Rock, Pop-Rock, Nu Disco,..."
1,SZA,Kill Bill,"December 9, 2022",Carter Lang & Rob Bisel,"SZA, Rob Bisel & Carter Lang",Nan,Sony Music Entertainment,"Pop, Alternative R&B, Alternative Pop, Psyched..."
2,Bizarrap & Shakira,"Bzrp Music Sessions, Vol. 53","January 11, 2023",Bizarrap,"Shakira, KEITYN, Santiago Alvarado & Bizarrap",Nan,The Orchard,"Pop, Deep House, Synth-Pop, Latin Urban, Latin..."
3,Sam Smith & Kim Petras,Unholy,"September 22, 2022","Blake Slatkin, Cirkut, ILYA, Jimmy Napes, Omer...","Blake Slatkin, Cirkut, ILYA, Jimmy Napes, Kim ...",Nan,Nan,"R&B, Pop, UK Pop, Electronic, TikTok, Dark Pop..."
4,Rema & Selena Gomez,Calm Down,"August 26, 2022",Andrevibez,"Andrevibez, Rema, Michael Hunter & Selena Gomez",Nan,Universal Music Group,"Pop, Remix, Nigerian Pidgin, Nigeria, Afrobeats"
5,"Metro Boomin, The Weeknd & 21 Savage",Creepin',"December 2, 2022","Metro Boomin, DaHeala, Peter Lee Johnson & joh...","Metro Boomin, 21 Savage, DaHeala, Peter Lee Jo...",Nan,Universal Music Group,"Pop, R&B, Rap, Downtempo, Pop Rap, Synth-Pop, ..."
6,Harry Styles,As It Was,"April 1, 2022",Tyler Johnson & Kid Harpoon,"Kid Harpoon, Tyler Johnson & Harry Styles","BLACKSTONE CREATIVE, Pulse Music Group & Conco...",Sony Music Entertainment,"Rock, Pop, Electronic, UK Pop, Dance Rock, Fun..."
7,RAYE Featuring 070 Shake,Escapism,"October 12, 2022",Mike Sabath,"RAYE, 070 Shake & Mike Sabath",Nan,Nan,"Rap, Pop, Trip-Hop, Alternative Pop, Pop Rap, ..."
8,David Guetta & Bebe Rexha,I'm Good (Blue),"July 31, 2012",T-Minus & Nikhil Seetharam,"Kendrick Lamar, T-Minus & Nikhil Seetharam",Nan,Universal Music Group,"Pop, Rap, Hip-Hop, Cloud Rap, Trap, Conscious ..."
9,Taylor Swift,Anti-Hero,"October 21, 2022",Jack Antonoff & Taylor Swift,Taylor Swift & Jack Antonoff,"Ducky Donath Music, Universal Music Group, Son...",Universal Music Group,"Pop, Pop-Rock, Electro-Pop, Electronic, Synth-..."
